<a href="https://colab.research.google.com/github/Stevens0856/OkadaVoiceChanger/blob/main/Okada_Voice_Changer_Colab_Unofficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align: center;">
  <span style="color: #00ffff;">OKADA VOICE CHANGER COLAB (UNOFFICIAL)</span>
</h1>


For mobile user may experience bugs and feature limitations.

In [ ]:
#=================Updated=================
# @title **Cell[1]** Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It can take some time to complete.**
import os
import time
import subprocess
import threading
import shutil
import base64
import codecs
import torch
import sys
import requests
from IPython.display import clear_output
from typing import Literal, TypeAlias
from google.colab import drive

# Fix some packages
!pip install pip==24.0.0 -q

Mode: TypeAlias = Literal["elf", "zip"]
mode:Mode="elf"

# Configs
%cd /content
Run_Cell=0

#@markdown Pilih Versi Okada Voice Changer / Choose Okada Voice Changer Version
version = "V1(new)" #@param ["V1(old)", "V1(new)", "V2"]

current_version_hash=None
latest_version_hash=None

# Check GPU
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")
    # sys.exit("No GPU available. Change runtime.")

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

if version == "V1(new)":
    print('\033[36m\033[1m\033[3mDownloading prebuilt executable...\033[0m')
    res = requests.get('https://api.github.com/repos/deiteris/voice-changer/releases/latest')
    release_info = res.json()

    for asset in release_info['assets']:
        if not asset['name'].startswith('voice-changer-linux-amd64-cuda.tar.gz'):
            continue
        download_url = asset['browser_download_url']
        !wget -q --show-progress {download_url}

    print('\033[32m\033[1m\033[3mUnpacking...\033[0m')
    !cat voice-changer-linux-amd64-cuda.tar.gz.* | tar xzf -
    !rm -rf voice-changer-linux-amd64-cuda.tar.gz.*
    print('\033[32m\033[1m\033[3mFinished unpacking!\033[0m')

    path = codecs.decode('ZZIPFreireFVB', 'rot_13')

    %cd $path

    # Additional
    !pip install python-dotenv pyngrok --quiet

    print('\033[32m\033[1m\033[3mSuccessfully downloaded and unpacked the binary!!\033[0m')
    # libportaudio2
    print('\033[36m\033[1m\033[3mInstalling libportaudio2...\033[0m')
    !apt-get -y install libportaudio2 -qq

    # Server Config
    %cd /content/$path

    from dotenv import set_key

    set_key('.env', "SAMPLE_MODE", "")

    Ready = True

    clear_output()
    print('\033[32m\033[1m\033[3mCell 1 Was Executed Completely\033[0m')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

elif version == "V1(old)":

    from IPython.display import clear_output

    # Define the URL and the destination paths
    url = 'https://huggingface.co/freyza/w-okada/resolve/main/w-okada.zip'
    zip_path = '/content/w-okada.zip'
    extract_path = '/content'

    # Download the zip file
    subprocess.run(['wget', '-q', '-O', zip_path, url], check=True)

    # Unzip the downloaded file to the specified directory
    subprocess.run(['unzip', '-o', '-q', zip_path, '-d', extract_path], check=True)

    # Remove the zip file after extraction
    subprocess.run(['rm', zip_path], check=True)

    # List the contents to verify
    subprocess.run(['ls', extract_path], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Install libportaudio2
    %cd /content/w-okada/server
    print('\033[36m\033[1m\033[3mInstalling libportaudio2...\033[0m')
    !apt-get -y install libportaudio2 -qq

    # Install additional dependencies
    print('\033[36m\033[1m\033[3mInstalling pre-dependencies...\033[0m')
    !python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/ -q
    !pip install faiss-gpu fairseq pyngrok --quiet
    !pip install pyworld --no-build-isolation --quiet

    # Install dependencies from requirements.txt
    print('\033[36m\033[1m\033[3mInstalling dependencies from requirements.txt...\033[0m')
    !pip install -r requirements.txt --quiet

    clear_output()
    print('\033[32m\033[1m\033[3mCell 1 Was Executed Completely\033[0m')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

elif version == "V2":

    from IPython.display import clear_output, Javascript

    work_dir = "/content"
    print("Downloading the latest vcclient... ")
    !curl -s -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/latest_hash.txt -o latest_hash.txt
    latest_version_hash = open('latest_hash.txt').read().strip()

    if mode == "elf":
        !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_latest_for_colab -o {work_dir}/vcclient_latest_for_colab
    elif mode == "zip":
        !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_latest_for_colab -o {work_dir}/vcclient_latest_for_colab.zip

    print("Download is done.")

    if current_version_hash != latest_version_hash and mode == "zip":
        print(f"Unzip vcclient to {latest_version_hash} ... ")
        !cd {work_dir} && unzip -q vcclient_latest_for_colab.zip -d {latest_version_hash}
        print(f"Unzip is done.")

    if mode == "elf":
        %cd {work_dir}
        !chmod 0700 vcclient_latest_for_colab
    elif mode == "zip":
        %cd {work_dir}/{latest_version_hash}/main
        !chmod 0700 main

    print("Installing modules... ", end="")
    !sudo apt-get install -y libportaudio2 > /dev/null 2>&1
    !pip install pyngrok > /dev/null 2>&1
    clear_output()
    print('\033[32m\033[1m\033[3mV2 Cell 1 Was Executed Completely\033[0m')

Cell 1 Was Executed Completely


In [ ]:
import codecs
#=======================Updated=========================

# @title **Cell[2]** Start Server **using ngrok**
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a while (~1-2 minutes)

# @markdown ---
# @markdown You'll need a ngrok account, but <font color=green>**it's free**</font> and easy to create!
# @markdown ---
# @markdown **1** - Create a <font color=green>**free**</font> account at [ngrok](https://dashboard.ngrok.com/signup) or **login with Google/Github account**\
# @markdown **2** - If you didn't logged in with Google/Github, you will need to **verify your e-mail**!\
# @markdown **3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and place it here:
Token = '2gPKwZyv68cWfRNex7aDDPFgCS7_3j4cuDS5sc4tNFXT946vF' # @param {type:"string"}
# @markdown **4** - *(optional)* Change to a region near to you or keep at United States if increase latency\
# @markdown `Default Region: ap - Asia/Pacific (Singapore)`
Region = "ap - Asia/Pacific (Singapore)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

#@markdown **5** - *(optional)* Other options:
ClearConsole = True  # @param {type:"boolean"}

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!
# ---------------------------------
if version == "V1(new)":
    %cd /content/MMVCServerSIO
    if not globals().get('Ready', False):
        print("Go back and run first cells.")
    else:
        from pyngrok import conf, ngrok
        MyConfig = conf.PyngrokConfig()
        MyConfig.auth_token = Token
        MyConfig.region = Region[0:2]
        conf.get_default().authtoken = Token
        conf.get_default().region = Region
        conf.set_default(MyConfig)

        import threading, time, socket
        PORT = 18888

        import json
        from pyngrok import ngrok
        from IPython.display import clear_output

        ngrokConnection = ngrok.connect(PORT)
        public_url = ngrokConnection.public_url
        set_key('.env', "ALLOWED_ORIGINS", json.dumps([public_url]))

        def wait_for_server():
            while True:
                time.sleep(0.5)
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                result = sock.connect_ex(('127.0.0.1', PORT))
                if result == 0:
                    break
                sock.close()
            if ClearConsole:
                clear_output()
            print("--------- SERVER READY! ---------")
            print("Your server is available at:")
            print(public_url)
            print("---------------------------------")

        threading.Thread(target=wait_for_server, daemon=True).start()

        !./$path

        ngrok.disconnect(ngrokConnection.public_url)
        print('\033[32m\033[1m\033[3mServer Telah Berhenti / Server Was Stopped\033[0m')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

elif version == "V1(old)":

    from pyngrok import conf, ngrok
    MyConfig = conf.PyngrokConfig()
    MyConfig.auth_token = Token
    MyConfig.region = Region[0:2]
    #conf.get_default().authtoken = Token
    #conf.get_default().region = Region
    conf.set_default(MyConfig)

    import subprocess, threading, time, socket, urllib.request
    PORT = 9999

    from pyngrok import ngrok
    ngrokConnection = ngrok.connect(PORT)
    public_url = ngrokConnection.public_url

    from IPython.display import clear_output

    def wait_for_server():
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', PORT))
            if result == 0:
                break
            sock.close()
        if ClearConsole:
            clear_output()
        print("--------- SERVER READY! ---------")
        print("Your server is available at:")
        print(public_url)
        print("---------------------------------")

    threading.Thread(target=wait_for_server, daemon=True).start()

    !python3 pengubahsuara.py \
      -p {PORT} \
      --https False \
      --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
      --content_vec_500_onnx pretrain/content_vec_500.onnx \
      --content_vec_500_onnx_on false \
      --hubert_base pretrain/hubert_base.pt \
      --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
      --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
      --nsf_hifigan pretrain/nsf_hifigan/model \
      --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
      --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
      --rmvpe pretrain/rmvpe.pt \
      --model_dir model_dir \
      --samples samples.json

    ngrok.disconnect(ngrokConnection.public_url)
    print('\033[32m\033[1m\033[3mServer Telah Berhenti / Server Was Stopped\033[0m')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

elif version == "V2":

    import time
    from pyngrok import ngrok
    from IPython.display import clear_output, display, Javascript, Audio

    Close_Ngrok=True
    Open_New_Tab=True
    PORT = 8003
    LOG_FILE = f"/content/LOG_FILE_{PORT}"

    def play_notification_sound(url):
        display(Audio(url=url, autoplay=True))
    Close_Ngrok = True
    Open_New_Tab = True

    # Start
    if mode == "elf":
        get_ipython().system_raw(f'LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib/x86_64-linux-gnu ./vcclient_latest_for_colab cui --port {PORT} --no_cui true >{LOG_FILE} 2>&1 &')
    elif mode == "zip":
        !LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib/x86_64-linux-gnu ./main cui --port {PORT} --no_cui true &

    if Close_Ngrok:
        ngrok.kill()

    ngrok.set_auth_token(Token)

    # Tunggu sampai server dimulai
    print('\033[31m\033[1m\033[3mTunggu sampai server dimulai\033[0m')
    time.sleep(130)

    main_tunnel = ngrok.connect(PORT)
    clear_output()

    if Open_New_Tab:
        display(Javascript(f'window.open("{main_tunnel.public_url}", "_blank");'))

    print("--------- SERVER LINK ---------")
    print('\033[32m\033[1m\033[3mServer Sudah Berjalan\033[0m')
    print("PUBLIC URL:", main_tunnel.public_url)
    print("---------------------------------")

--------- SERVER READY! ---------
Your server is available at:
https://2bd9-34-16-138-214.ngrok-free.app
---------------------------------
2024-12-13 00:53:31,247 INFO     [main] --------
2024-12-13 00:53:31,248 INFO     [main] The server is listening on http://127.0.0.1:18888/
2024-12-13 00:53:31,248 INFO     [main] --------
2024-12-13 00:54:16,273 INFO     [MMVC_Namespace] Connected SID: kzEmF9lyYCf9wok6AAAB
2024-12-13 00:56:04,740 INFO     [MMVC_Rest_Fileuploader] paramDict
2024-12-13 00:56:04,740 INFO     [VoiceChangerManager] FILE: LoadModelParamFile(name='zetaTest.pth', kind='rvcModel', dir='')
2024-12-13 00:56:04,740 INFO     [VoiceChangerManager] Moving /tmp/tmpes66vydm/upload_dir/zetaTest.pth -> model_dir/0/zetaTest.pth
2024-12-13 00:56:04,740 INFO     [RVCModelSlotGenerator] RVC:: slotInfo.modelFile zetaTest.pth
voice_changer/RVC/RVCModelSlotGenerator.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default

ERROR:    Traceback (most recent call last):
  File "asyncio/runners.py", line 44, in run
  File "asyncio/base_events.py", line 636, in run_until_complete
  File "asyncio/base_events.py", line 603, in run_forever
  File "asyncio/base_events.py", line 1909, in _run_once
  File "asyncio/events.py", line 80, in _run
  File "main.py", line 140, in main
  File "main.py", line 81, in runServer
  File "uvicorn/server.py", line 68, in serve
  File "contextlib.py", line 142, in __exit__
  File "uvicorn/server.py", line 332, in capture_signals
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "starlette/routing.py", line 700, in lifespan
  File "uvicorn/lifespan/on.py", line 137, in receive
  File "asyncio/queues.py", line 159, in get
asyncio.exceptions.CancelledError

Server Telah Berhenti / Server Was Stopped


In [ ]:
import os
import sys
import json
import requests
import codecs

if version == "V1(old)":

    #@title **[Optional Cell For V1 Old Okada Voice Changer]** Upload a voice model (Run this before running the Voice Changer)
    #@markdown Find your model here [voice-models](https://voice-models.com/)
    # @markdown ---
    model_slot = "0" #@param ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199']

    !rm -rf model_dir/$model_slot
    #@markdown **[Optional]** Add an icon to the model (Kosongin aja gapapa / It's okay to leave it blank)
    icon_link = ""  #@param {type:"string"}
    icon_link = '"'+icon_link+'"'
    !mkdir model_dir
    !mkdir model_dir/$model_slot
    #@markdown Put your model's download link here `(must be a zip file and don't use GPT-SoVITS Model)` only supports **huggingface.co** & **google drive**<br>
    model_link = "https://huggingface.co/megaaziib/my-rvc-models-collection/resolve/main/kobo.zip"  #@param {type:"string"}

    if model_link.startswith("https://www.weights.gg") or model_link.startswith("https://weights.gg"):
        print("Links from weights.gg is no longer supported.")
        sys.exit()
    elif model_link.startswith("https://drive.google.com"):
        model_link = '"'+model_link+'"'
        !gdown $model_link --fuzzy -O model.zip
        print("Model from Drive")
    elif model_link.startswith("https://huggingface.co"):
        model_link = model_link
        model_link = '"'+model_link+'"'
        !curl -L $model_link > model.zip
        print("Model from huggingface Link")
    else:
        model_link = model_link
        model_link = '"'+model_link+'"'
        !curl -L -O $model_link
        !mv ./*.pth model_dir/$model_slot/
        print('Model(.pth) or a direct model link.')

    # Conditionally set the iconFile based on whether icon_link is empty
    if icon_link == '""':
        iconFile = ""
        print("icon_link is empty, so no icon file will be downloaded.")
    else:
        iconFile = "icon.png"
        !curl -L $icon_link > model_dir/$model_slot/icon.png

    !unzip model.zip -d model_dir/$model_slot

    !mv model_dir/$model_slot/*/* model_dir/$model_slot/
    !rm -rf model_dir/$model_slot/*/
    !rm -rf model.zip
    #@markdown **Model Voice Conversion Setting**
    Tune = 0  #@param {type:"slider",min:-24,max:24,step:1}
    Index = 0  #@param {type:"slider",min:0,max:1,step:0.1}

    param_link = ""
    if param_link == "":
        paramset = requests.get("https://pastebin.com/raw/P6ar4baa").text
        exec(paramset)

    clear_output()
    print("\033[93mModel with the name of "+model_name+" has been Imported to slot "+model_slot)

elif version == "V1(new)":
    print('\033[31m\033[1m\033[3mV1 New Belum Support / V1 New is not supported\033[0m')
elif version == "V2":
    print('\033[31m\033[1m\033[3mV2 Belum Support / V2 is not supported\033[0m')